In [ ]:
import os

def delete_videos(dir_path):
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".mp4"):
                os.remove(os.path.join(root, file))

dir_path = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/results-shown/result_shown_nba_5k_7s"
delete_videos(dir_path)

In [10]:
from format_data_kinetics_tf import clean_directory, format_dataset_kinetics
from split_dataset import copy_and_split_dataset

dst_dir = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/nba_results_hidden_1k_4s"
# test_dir = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/test-sets/result-shown/result_shown_test_nba_1k_8s"

# copy_and_split_dataset(
#     train_val_dir="/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/results-shown/result_shown_nba_52k_7s",
#     test_dir=test_dir,
#     dst_dir=dst_dir,
#     num_files_to_copy=5000,
# )

clean_directory(dst_dir)
# format_dataset_kinetics(dst_dir)

100%|██████████| 1769/1769 [00:27<00:00, 64.12it/s]
